In [1]:
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd

from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display



data_file = "cma_es.pkl"
features = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation']
df = pd.read_pickle(data_file)



cs = ConfigurationSpace({
    "elitist": list(map(str, df['elitist'].unique())), 
    "mirrored" : list(map(str, df['mirrored'].unique())), 
    "base_sampler" : list(map(str, df['base_sampler'].unique())), 
    "weights_option" : list(map(str, df['weights_option'].unique())), 
    "local_restart" : list(map(str, df['local_restart'].unique())), 
    "step_size_adaptation" : list(map(str, df['step_size_adaptation'].unique()))
})

print(cs)
print( df['dim'].unique())
cmaes_explainer = explainer(None, 
                 cs , 
                 dims = [5,30],#, 10, 20, 40 
                 fids = np.arange(1,25), #,5
                 iids = len( df['iid'].unique()), #20 
                 reps = len( df['seed'].unique()), 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = {},
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)


cmaes_explainer.load_results(data_file)
#for feature in features:
#    cmaes_explainer.df[feature] = cmaes_explainer.df[feature].astype("category")
df = cmaes_explainer.performance_stats()
display(df.style.bar(cmap='viridis'))

#cmaes_explainer.plot(partial_dependence=False, best_config=False)
#print(de_explainer.stats[5])
#df = de_explainer.stats[5]
#print(df.to_latex(index=False,
#                  float_format="{:.2f}".format,
#                  multicolumn_format = "c"
#))  


/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/bas/repos/ioh-explainer/env/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
IProgress not found. Please update jupyter and ipywidgets. See

Configuration space object:
  Hyperparameters:
    base_sampler, Type: Categorical, Choices: {sobol, gaussian, halton}, Default: sobol
    elitist, Type: Categorical, Choices: {True, False}, Default: True
    local_restart, Type: Categorical, Choices: {nan, IPOP, BIPOP}, Default: nan
    mirrored, Type: Categorical, Choices: {mirrored pairwise, nan, mirrored}, Default: mirrored pairwise
    step_size_adaptation, Type: Categorical, Choices: {csa, psr}, Default: csa
    weights_option, Type: Categorical, Choices: {default, equal, 1/2^lambda}, Default: default

[30  5]


In [3]:
from IPython.display import display

#df = de_explainer.behaviour_stats()
#df.style.bar(cmap='viridis')

#do it for f1 and f5

df = cmaes_explainer.behaviour_stats(per_fid=False)
display(df.style.bar(cmap='viridis'))

df = cmaes_explainer.behaviour_stats(per_fid=True)
display(df.style.bar(cmap='viridis'))

[0 1 2 3 4 5 6 7 8 9]
[1 4 2 3 5]
